In [4]:
# Import numpy and scipy libraries
import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve

# Define the boundary value problem
# y'' + p(x)y' + q(x)y = g(x) on [a,b]
# y(a) = alpha, y(b) = beta
a = 0  # left endpoint
b = 1  # right endpoint
alpha = 0  # left boundary value
beta = 0  # right boundary value
N = 10  # number of subintervals
h = (b - a) / N  # subinterval length
x = np.linspace(a, b, N + 1)  # grid points


# Define the coefficient functions
def p(x):
    return -2 * x


def q(x):
    return 2


def g(x):
    return -4 * x * (x - 1)


# Define the basis functions
def phi(i, z):
    # Linear hat function
    if i == 0:
        return (x[i + 1] - z) / (h)
    elif i == N:
        return (z - x[i - 1]) / (h)
    else:
        return (z - x[i - 1]) / (h) + (x[i + 1] - z) / (h)


def phi_prime(i, z):
    # Derivative of linear hat function
    if i == 0:
        return -1 / h
    elif i == N:
        return 1 / h
    else:
        return -1 / h + 1 / h


# Construct the linear system
A = np.zeros((N + 1, N + 1))  # coefficient matrix
b = np.zeros(N + 1)  # right-hand side vector

# Loop over the subintervals
for i in range(N + 1):
    # Loop over the grid points
    for j in range(N + 1):
        # Compute the matrix entries using the symmetric weak form
        A[i, j] = (
            phi_prime(i, x[j]) * phi_prime(j, x[j])
            + 0.5 * (p(x[i]) + p(x[j])) * phi_prime(i, x[j]) * phi(j, x[j])
            + 0.5 * (q(x[i]) + q(x[j])) * phi(i, x[j]) * phi(j, x[j])
        )
        # Apply the boundary conditions
        if i == 0:
            A[i, j] = 0
            if j == 0:
                A[i, j] = 1
        elif i == N:
            A[i, j] = 0
            if j == N:
                A[i, j] = 1
    # Compute the right-hand side entries
    b[i] = g(x[i]) * phi(i, x[i])
    # Apply the boundary conditions
    if i == 0:
        b[i] = alpha
    elif i == N:
        b[i] = beta

# Convert A to a sparse matrix
A = diags(A.diagonal()) + diags(A.diagonal(1), 1) + diags(A.diagonal(-1), -1)

# Solve the linear system
y = spsolve(A, b)

# Print the solution
print("The solution is:")
print(y)

The solution is:
[4.16333634e-17 3.00000000e-02 6.00000000e-02 7.00000000e-02
 8.00000000e-02 9.00000000e-02 8.00000000e-02 7.00000000e-02
 6.00000000e-02 3.00000000e-02 0.00000000e+00]


c:\Users\faliq\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:229: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
